<a name="POS-1"></a>

In [ ]:
from gensim.models import Word2Vec
import re

def return_documents():
    """
    Returns a list of documents, where each document is a string
    """
    from sklearn.datasets import fetch_20newsgroups
    dataset = fetch_20newsgroups()
    corpus = dataset.data
    return corpus

def tokenize_and_tag_documents(documents, nlp, sep_char="||||"):
    """
    Returns a list of lists of tokens. Each token has been 
    concatenated with its part of speech
    """
    for doc in nlp.pipe(documents, parse=False, entity=False):
        yield [word.lower_ + sep_char + word.pos_ for word in doc]
        

def build_model(tokenized_docs):
    """
    Returns a gensim Word2Vec model trained on our corpus
    """
    return Word2Vec(list(tokenized_docs), size=128)


documents = return_documents()
tokenized_and_tagged_documents = tokenize_and_tag_documents(documents, nlp)
model = build_model(tokenized_and_tagged_documents)


def return_alts(word):
    pattern = re.compile("^" + word + "\|\|\|\|")
    return list(filter(lambda x: bool(pattern.match(x)), model.wv.vocab.keys()))

if model:
    print(model.most_similar('real||||ADJ'))
    print()
    print(model.most_similar('real||||ADV'))

<a name="POS-2">Solution for POS Exercise 2</a>

In [ ]:
from sklearn.utils import shuffle
class PerceptronClassifier(object):
    def __init__(self):
        self.weights = None
        self.bias = None
        self.iter = 0

    def fit(self, X, y, epochs=100):
        """Fits self.weights, self.biases """
        self.initialize(X)
        
        for _ in range(epochs):
            for row, label in zip(X, y):
                prediction = self.predict(row)
                self.update(prediction, label, row)
                self.iter += 1
            X, y = shuffle(X, y) #important to reshuffle to avoid getting trapped
            
    def initialize(self, X):
        self.bias = 0
        self.weights = np.zeros(X.shape[1])#[:, np.newaxis]
                    
    def update(self, prediction, label, row):
        """Updates weights and biases based on the ground truth label
        and the row"""
        
        if prediction*label <= 0:
            update = label *  row            
            self.weights += update
            self.bias += label
        
    def predict_score(self, x):
        """Generates scores of "x". Uses self.weights and self.bias"""
        return np.dot(x, self.weights) + self.bias
    
    def predict(self, x):
        """ Converts prediction scores to 1s ands 0s."""
        predictions = np.where(self.predict_score(x) > 0, 1, -1)
        return predictions